Test ORCID matching

In [1]:
from pathlib import Path

from fastjsonschema import compile as compile_schema, JsonSchemaException
import jsonref as json

SCHEMA_FOLDER = Path("../schema.igsn.org/json/registration/0.1/").absolute()

def test_definition(schema_file, defn, examples):
    """
    Test a definition in our JSON schema using some examples
    
    Examples take the form of a Python object and the expected
    validation outcome (i.e. True or False)
    
    Parameters:
        schema_file - a root file location to test schemas against
        ref - a reference to the fragment of the schema that you want to test
        examples - the example data you want to test
    """
    schema = json.loads(
        f'{{ "$ref": "#/definitions/{defn}" }}', 
        base_uri=(SCHEMA_FOLDER / schema_file).as_uri()
    )
    validate = compile_schema(schema)
    for obj, expected in examples:
        try:
            validate(obj)
            if not expected:
                raise AssertionError(f'Object {obj} unexpectedly validated')
        except JsonSchemaException as err:
            if expected:
                raise AssertionError(
                    f'Object {obj} failed to validate. Error is {err.message()}'
                )

Test ORCIDS

In [2]:
examples = [
    ({"kind": "orcid", "id": "0234-4568-7895-1655"}, True),
    ({"kind": "orcid", "id": "0234 4568 7895 1655"}, True),
    ({"kind": "orcid", "id": "0234-XXXXD-7895-1655"}, False),
    ({"kind": "orcid", "id": "0234-4581-7895-1655-4561"}, False),
    ({"kind": "orcid", "id": "http://orcid.com/0234-4568-7895-1655"}, True),
    ({"kind": "orcid", "id": "http://orcid.com/0234 4568 7895 1655"}, False),
    ({"kind": "orcid", "id": "http://orcid.com/0234-XXXX-7895-1655"}, False),
    ({"kind": "orcid", "id": "http://orcid.com/0234-4581-7895-1655-4561"}, False),
    ({"kind": "orcid", "id": "https://orcid.com/0234-4568-7895-1655"}, True),
    ({"kind": "orcid", "id": "https://orcid.com/0234 4568 7895 1655"}, False),
    ({"kind": "orcid", "id": "https://orcid.com/0234-XXXX-7895-1655"}, False),
    ({"kind": "orcid", "id": "https://orcid.com/0234-4581-7895-1655-4561"}, False),
]
test_definition('identifiers.json', 'orcid', examples)

Test ResearcherID

In [3]:
examples = [
    ({"kind": "researcherId", "id": "X-1238-2347"}, True),
    ({"kind": "researcherId", "id": "S-4816-2540"}, True),
    ({"kind": "researcherID", "id": "S-4816-2540"}, False),
    ({"kind": "researcherId", "id": "S-XXXX-7895-1655"}, False),
    ({"kind": "researcherId", "id": "S-4581-7895-1655"}, False)
]
test_definition('identifiers.json', 'researcherId', examples)

Test merged schemas

In [20]:
SCHEMA_FILE = 'identifiers.json'

TEST_SCHEMA = json.loads("""
    {
        "type": "array",
        "items": {
            "anyOf": [
                { "$ref": "identifiers.json#/definitions/viaf" },
                { "$ref": "identifiers.json#/definitions/researcherId" },
                { "$ref": "identifiers.json#/definitions/isni" },
                { "$ref": "identifiers.json#/definitions/orcid" }
            ]
        }
    }
    """,
    base_uri=(SCHEMA_FOLDER / SCHEMA_FILE).as_uri()
)

examples = [
    ([
        {'kind': 'orcid', 'id': '0000-0002-4553-9697'},
        {'kind': 'researcherId', 'id': 'I-4625-2012'}
    ], True),
    ([
        {'kind': 'orcid', 'id': '0000-0002-4553-9697'}
    ], True),
    ([
        {'kind': 'researcherId', 'id': 'I-4625-2012'}
    ], True),
    ([], True)
]

def test_multiid(example):
    """
    Test a definition in our JSON schema using some examples
    
    Examples take the form of a Python object and the expected
    validation outcome (i.e. True or False)
    
    Parameters:
        schema_file - a root file location to test schemas against
        ref - a reference to the fragment of the schema that you want to test
        examples - the example data you want to test
    """
   
    validate = compile_schema(TEST_SCHEMA)
    obj, expected = example
    try:
        validate(obj)
        if not expected:
            raise AssertionError(f'Object {obj} unexpectedly validated')
    except JsonSchemaException as err:
        if expected:
            raise AssertionError(
                f'Object {obj} failed to validate. Error is {err.message}'
            )
            
for example in examples:
    test_multiid(example)

Test IGSN registration data

In [21]:
# Design some examples
examples = [
    ({
        "igsn": "https://igsn.org/FOOTEST234",
        "registrant": {
            "name": "Jess Robertson", 
            "identifiers": [
                {'kind': 'orcid', 'id': '0000-0002-4553-9697'},
                {'kind': 'researcherId', 'id': 'I-4625-2012'}
            ]
        }
    }, True),
    ({"igsn": "https://igsn.org/FOOTEST234"}, False),
    ({"registrant": {"name": "jess"}}, False),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": "jess"
    }, False),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": {"name": "jess"}
    }, True),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": {"name": "jess", 'identifiers': []}
    }, True),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": {"name": "jess"},
        "related": [],
    }, True),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": {"name": "jess"},
        "related": ["foo"],
    }, False),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": {"name": "jess"},
        "related": [
            {"kind": "igsn", "id": "FOOTEST235", "relationship": "isPartOf"}
        ],
    }, True),
    ({
        "igsn": "https://igsn.org/FOOTEST234", 
        "registrant": {"name": "jess"},
        "related": [
            {"kind": "igsn", "id": "FOOTEST235", "relationship": "isPartOf"},
            {"kind": "orcid", "id": "0000-0002-4553-9697", "relationship": "isCitedBy"}
        ],
    }, True),
]

# Load up the actual schema
schema_folder = Path("../schema.igsn.org/json/registration/0.1/").absolute()
schema_file = schema_folder /'registry.json'
with open(schema_file, 'r', encoding='utf-8') as src:
    # Here we're using jsonref to dereference local files for dev purposes
    # in production these will all need to be replaced with 
    # dereferencable URIs
    schema = json.load(src, base_uri=schema_file.as_uri())
    validate = compile_schema(schema)

def test_igsn_registration(example):
    "Test some IGSN inputs"
    # Validate against examples
    obj, expect_to_pass = example
    try:
        validate(obj)
        if not expect_to_pass:
            raise AssertionError(f'Object {obj} unexpectedly validated')
    except JsonSchemaException as err:
        if expect_to_pass:
            raise AssertionError(f'Object {obj} failed to validate. Error is {err.message}')
                
for ex in examples:
    test_igsn_registration(ex)